## Lab Test 2

#### Problem:
A tool for testing internet bandwidth outputs the following into a text file for each hour:  
Day, Hour, Bandwidth in MB/s  
1,0,100.34  
1,1,123.89  
.  
.  
.  
1,22,96.77  
1,23,49.28  

The numbers 0-23 indicate the hour of the day in 24-hour format. This is followed by the data for 
the next day and so on. Assume that your processing begins on data with complete days.

Write Spark code to find the following:
1. Average i th hour bandwidth across all the days given i.
[4M]
2. Number of days when the i
th hour bandwidth was higher than the average i
th hour bandwidth.
3. Best average Quarter. Q1: 0-5, Q2: 6-11, Q3: 12-17, Q4: 18-23. Best Quarter is the one that had 
the highest average bandwidth across all days.
4. The hour of the day that had the best increase and the worst fall compared to the previous hour.


In [102]:
 L= [(1,0,112.36),
(1,1,143.89),
(1,2,129.43),
(1,3,115.57),
(1,4,101.86),
(1,5,99.36),
(1,6,120.00),
(1,7,135.72),
(1,8,99.53),
(1,9,121.22),
(1,10,115.68),
(1,11,122.73),
(1,12,145.65),
(1,13,144.27),
(1,14,140.79),
(1,15,146.39),
(1,16,122.23),
(1,17,112.65),
(1,18,112.35),
(1,19,100.65),
(1,20,99.75),
(1,21,140.32),
(1,22,153.26),
(1,23,180.99),
(2,0,112.36),
(2,1,143.89),
(2,2,129.43),
(2,3,115.57),
(2,4,101.86),
(2,5,99.36),
(2,6,120.00),
(2,7,135.72),
(2,8,99.53),
(2,9,121.22),
(2,10,115.68),
(2,11,122.73),
(2,12,145.65),
(2,13,144.27),
(2,14,140.79),
(2,15,146.39),
(2,16,122.23),
(2,17,112.65),
(2,18,112.35),
(2,19,100.65),
(2,20,99.75),
(2,21,140.32),
(2,22,153.26),
(2,23,180.99),
(3,0,112.36),
(3,1,143.89),
(3,2,129.43),
(3,3,115.57),
(3,4,101.86),
(3,5,99.36),
(3,6,120.00),
(3,7,136.72),
(3,8,99.53),
(3,9,121.22),
(3,10,115.68),
(3,11,122.73),
(3,12,145.65),
(3,13,144.27),
(3,14,140.79),
(3,15,146.39),
(3,16,122.23),
(3,17,112.65),
(3,18,112.35),
(3,19,100.65),
(3,20,99.75),
(3,21,140.32),
(3,22,153.26),
(3,23,180.99)]

In [1]:
## setup
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
from pyspark import SparkContext, SparkConf

conf = pyspark.SparkConf().setAppName('LabTest2').setMaster('local')
sc = pyspark.SparkContext(conf=conf)

In [104]:
## Load data
rdd = sc.textFile("data/bandwidths.txt")
rdd.take(2)

['1,0,112.36', '1,1,143.89']

In [105]:
## formatting record
data = rdd.map(lambda a: [float(x) for x in a.split(",")])

In [106]:
data.take(1)

[[1.0, 0.0, 112.36]]

### 1. Average of i th hour bandwidth across all the days given i.

In [20]:
def hrAvgBW(i):
    t1 = data.filter(lambda a: a[1]==i)
    t2 = t1.map(lambda a: (a[1],(a[2],1))).reduceByKey(lambda a,b : (a[0]+b[0],a[1]+b[1]))
    t3 = t2.map(lambda a: (a[0],a[1][0]/a[1][1]))
    return(t3.collect()[0][1])

In [116]:
print("Average of 7th hour bandwidth across all the days:")
print(hrAvgBW(7))

Average of 7th hour bandwidth across all the days:
136.38666666666666


### 2. Number of days when the ith hour bandwidth was higher than the average ith hour bandwidth.

In [111]:
def ndOavg(i):
    avgi = hrAvgBW(i)
    c = data.filter(lambda a: a[1]==i and a[2]>avgi ).count()
    return(c)

In [113]:
print("Number of days when the 7th hour bandwidth was higher than the average 7th hour bandwidth:")
print(ndOavg(7))

Number of days when the 7th hour bandwidth was higher than the average 7th hour bandwidth:
1


### 3. Best average Quarter. Q1: 0-5, Q2: 6-11, Q3: 12-17, Q4: 18-23. Best Quarter is the one that had the highest average bandwidth across all days.

In [31]:
def quartermap(hr):
    if hr>=0 and hr<=5:
        return 1
    if hr>=6 and hr<=11:
        return 2
    if hr>=12 and hr<=17:
        return 3
    if hr>=18 and hr<=23:
        return 4

In [84]:
def bestQ():
    t1 = data.map(lambda a: (quartermap(a[1]),(a[2],1)))
    t2 = t1.reduceByKey(lambda a,b: (a[0]+b[0],a[1]+b[1]))
    t3 = t2.map(lambda a: (a[0], a[1][0]/a[1][1]))
    bq = t3.reduce(lambda a,b: a if a[1]>b[1] else b)
    return(bq[0])

In [115]:
print("Best Average Quarter:")
print("Q",bestQ())

Best Average Quarter:
Q 3


### 4. The hour of the day that had the best increase and the worst fall compared to the previous hour.

In [117]:
## econding hour to a timestamp 
t1 = data.map(lambda a: (a[1]+(a[0]-1)*24 , a) ) 

## producing a new record to compare with next timestamp, for each record
t2 =t1.flatMap(lambda a: [(a[0],[a[0],a[1][1],a[1][2]]),(a[0]+1,[a[0],a[1][1],a[1][2]])] ) 

## calculating change for each day compered to its previous day
t3 = t2.reduceByKey(lambda a,b: (b[1], b[2]-a[2]) if a[0]<b[0] else (a[1], a[2]-b[2])).filter(lambda a: len(a[1])==2)

t3.collect()

[(1.0, (1.0, 31.529999999999987)),
 (2.0, (2.0, -14.45999999999998)),
 (3.0, (3.0, -13.860000000000014)),
 (4.0, (4.0, -13.709999999999994)),
 (5.0, (5.0, -2.5)),
 (6.0, (6.0, 20.64)),
 (7.0, (7.0, 15.719999999999999)),
 (8.0, (8.0, -36.19)),
 (9.0, (9.0, 21.689999999999998)),
 (10.0, (10.0, -5.539999999999992)),
 (11.0, (11.0, 7.049999999999997)),
 (12.0, (12.0, 22.92)),
 (13.0, (13.0, -1.3799999999999955)),
 (14.0, (14.0, -3.480000000000018)),
 (15.0, (15.0, 5.599999999999994)),
 (16.0, (16.0, -24.159999999999982)),
 (17.0, (17.0, -9.579999999999998)),
 (18.0, (18.0, -0.30000000000001137)),
 (19.0, (19.0, -11.699999999999989)),
 (20.0, (20.0, -0.9000000000000057)),
 (21.0, (21.0, 40.56999999999999)),
 (22.0, (22.0, 12.939999999999998)),
 (23.0, (23.0, 27.730000000000018)),
 (24.0, (0.0, -68.63000000000001)),
 (25.0, (1.0, 31.529999999999987)),
 (26.0, (2.0, -14.45999999999998)),
 (27.0, (3.0, -13.860000000000014)),
 (28.0, (4.0, -13.709999999999994)),
 (29.0, (5.0, -2.5)),
 (30.0, (6

In [118]:
## locating the highest change
best_increase = t3.reduce(lambda a,b: a if a[1][1]>b[1][1] else b)
print("Hour of the day that had the best increase:")
print(best_increase[1][0])

Hour of the day that had the best increase:
21.0


In [119]:
## Calculating the lowest change
best_fall = t3.reduce(lambda a,b: a if a[1][1]<b[1][1] else b)
print("Hour of the day that had the worst fall:")
print(best_fall[1][0])

Hour of the day that had the worst fall:
0.0
